In [2]:
import tensorflow as tf
import os

In [3]:
# 模拟一下同步synchronization 先处理数据，然后才能取数据训练
# tensorflow当中，运行操作有依赖性

# 1. 定义队列
Q = tf.FIFOQueue(3, tf.float32)

enq_many = Q.enqueue_many([[0.1, 0.2, 0.3]])

#2. 定义一些处理数据的逻辑，取数据过程：取数据， +1， 入队列
out_q = Q.dequeue()

data = out_q + 1

en_q = Q.enqueue(data)
print(Q.size())


Tensor("fifo_queue_Size:0", shape=(), dtype=int32)


In [4]:
with tf.Session() as sess:
    #初始化队列
    sess.run(enq_many)
    
    # 处理数据
    for i in range(1):
        sess.run(en_q)
        
    # 训练数据
    for i in range(Q.size().eval()):
        print(sess.run(Q.dequeue()))

0.2
0.3
1.1


In [ ]:
# 模拟异步asynchronization子线程: 存入样本，主线程: 读取样本

# 1. 定义一个队列， 1000
Q = tf.FIFOQueue(1000, tf.float32)

# 2. 定义要做的事 循环  值 +1 , 放入队列当中
var = tf.Variable(0.0)

# 实现一个自增 tf.assign_add
data = tf.assign_add(var, tf.constant(1.0))

en_q = Q.enqueue(data)

# 定义队列管理器op，指定多少个子线程，子线程干什么事
qr = tf.train.QueueRunner(Q, enqueue_ops=[en_q] *2 )

# 初始化变量op
init_op = tf.global_variables_initializer()


with tf.Session() as sess:
    # 初始化变量
    sess.run(init_op)
    
    # 开启线程管理器
    coord = tf.train.Coordinator()
    
    # 真正开启子线程
    threads = qr.create_threads(sess, coord=coord, start=True)
    
    # 主线程，不断读取数据训练
    for i in range(10):
        print(sess.run(Q.dequeue()))
    
    #回收线程
    coord.request_stop()
    
    coord.join(threads)
    

In [5]:
FLAGS = tf.app.flags.FLAGS

In [6]:
tf.app.flags.DEFINE_string("csv_dir", "./data/csv_data", "文件的目录")

In [7]:
tf.app.flags.DEFINE_string("dog_dir", "./data/dog", "文件的目录")

In [8]:
tf.app.flags.DEFINE_string("f", "", "kernel")

In [9]:
print(FLAGS.csv_dir)

./data/csv_data


In [ ]:
"""
读取csv文件
filelist：文件路径 + 名字的列表
"""

# 构造文件列表 路径+文件名
file_name = os.listdir(FLAGS.csv_dir)
filelist = [os.path.join(FLAGS.csv_dir, i) for i in file_name]


#1. 构造文件的队列
file_queue = tf.train.string_input_producer(filelist)

#2. 构造csv阅读器 读取队列数据 （按一行）
reader = tf.TextLineReader()
key, value = reader.read(file_queue)

# 3. 对每行内容解码
# record_defaults: 指定每一个样本的每一列的类型，指定默认值[["None"], [4.0]]
records = [["None"],["None"] ,[3]]

example, label, value = tf.decode_csv(value, record_defaults=records)

# 4. 想要读取多个数据，就需要批处理
example_batch, label_batch = tf.train.batch([example, label],
                                            batch_size=9, 
                                            num_threads=1, 
                                            capacity=9) 
# print(example, label) 
# print(temp) 
print(example_batch, label_batch)

with tf.Session() as sess:
    # 定义一个线程协调器
    coord = tf.train.Coordinator()
    
    # 开启读文件线程
    threads = tf.train.start_queue_runners(sess, coord=coord)
    
    # 打印读取的内容
    print(sess.run([example_batch, label_batch]))
    
    #回收子线程
    coord.request_stop()
    coord.join(threads)
    

In [10]:
# 读取狗图片并转换成张量

filelist= [os.path.join(FLAGS.dog_dir, i) for i in os.listdir(FLAGS.dog_dir)]

#1. 构造文件队列
file_queue = tf.train.string_input_producer(filelist)

# 2. 构造阅读器去读取图片内容（默认读取一张图片）
reader = tf.WholeFileReader()
key, value = reader.read(file_queue)

print(value)

# 3. 对读取的图片数据进行解码
image = tf.image.decode_jpeg(value)

print(image)

# 4. 处理图片大小（统一大小）, resize后，图片数据 uint8 --> float32
image_resize= tf.image.resize_images(image, [200, 200]) 
print(image_resize) 
# 一定要把样本的形状固定 [200, 200,3], 在批处理时要求所有数据形状必须定义
image_resize.set_shape([200,200,3])
print(image_resize)

# 5. 批处理
image_batch = tf.train.batch([image_resize],
                             batch_size=20,
                            num_threads=1,
                            capacity=20)

print(image_batch)

W0204 02:56:49.976355 139770633439040 deprecation.py:323] From <ipython-input-10-51116c11600a>:6: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0204 02:56:49.995707 139770633439040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/input.py:278: input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0204

Tensor("ReaderReadV2:1", shape=(), dtype=string)
Tensor("DecodeJpeg:0", shape=(?, ?, ?), dtype=uint8)
Tensor("resize/Squeeze:0", shape=(200, 200, ?), dtype=float32)
Tensor("resize/Squeeze:0", shape=(200, 200, 1), dtype=float32)
Tensor("batch:0", shape=(20, 200, 200, 1), dtype=float32)


In [11]:
with tf.Session() as sess:
    #线程协调器
    coord = tf.train.Coordinator()
    
    # 开启读文件的线程
    threads = tf.train.start_queue_runners(sess, coord=coord)
    
    print(sess.run(key))
    print(sess.run(image).shape)
    
    coord.request_stop()
    coord.join()

W0204 02:56:54.330092 139770633439040 deprecation.py:323] From <ipython-input-11-f75bdb47c561>:6: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


b'./data/dog/dog.85.jpg'
(218, 199, 3)


InvalidArgumentError: Shape mismatch in tuple component 0. Expected [200,200,1], got [200,200,3]
	 [[{{node batch/fifo_queue_enqueue}}]]